In [2]:
import pandas as pd

import great_expectations as gx
from great_expectations.core.expectation_configuration import ExpectationConfiguration

from ydata_profiling import ProfileReport
from ydata_profiling.config import Settings
from ydata_profiling.model import BaseDescription, expectation_algorithms
from ydata_profiling.model.handler import Handler
from ydata_profiling.utils.dataframe import slugify
from ydata_profiling.expectations_report import ExpectationsReport
from great_expectations.checkpoint import SimpleCheckpoint

from typing import Any, Optional

import pandas as pd
from visions import VisionsTypeset
import re
import os

In [ ]:
full_path = os.getcwd()
df = pd.read_csv(full_path.partition('notebooks')[0] + 'data/01_raw/Customers.csv')
df

,NewId,CustomerSince,CustomerStatus,DateOfBirth,EmploymentStatus,Gender,MaritalStatus,Placebrth,CustType,Nationality,...,Habliter,Province,District,LegalDocName1Id,LegalDocName1IdDescription,LegalIssDate,LegalExpDate,LegalIssAuth,AMLRiskRating,YrNetMonthlyIn
0,1,2012-05-23,Private Client - Standard,NaN,NaN,FEMALE,NaN,NaN,NaN,Portugal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevado,NaN
1,2,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medio,NaN
2,3,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medio,NaN
3,4,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevado,NaN
4,5,2013-11-14,Private Client - Standard,2012-07-01,MB,MALE,OTHER,NACALA PORTO,NaN,Mozambique,...,NaN,NAMPULA,NACALA-PORTO,36.0,Licenca/Alvara,20121113,20501113,NAMPULA,Medio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122207,122208,2016-11-29,Private Client - Standard,1973-01-16,EMPLOYED,MALE,DIVORCED,ISRAEL,RETAIL,Israel,...,Curso Superior,CABO DELGADO,MAHATE,2.0,DIRE,20240726ý20250520ý20250520,20250725ý20280519ý20280519,MIGRACAOýMAIRRO MAHATEýREEF INVESTIMENTOS,Baixo,346500.0
122208,122209,2021-07-23,Private Client - Standard,1978-01-10,EMPLOYED,MALE,SINGLE,ZIMBABWE,NaN,Zimbabwe,...,NaN,MAPUTO,MATOLA 'A',3.0,Passaporte,20170614,20270613,REPUBLIC OF ZIMBABWE,Medio,3000.0
122209,122210,2017-12-04,Private Client - Standard,1964-01-16,MB,MALE,DIVORCED,HLATHIKHULU,RETAIL,South Africa,...,NaN,NaN,NaN,3.0,Passaporte,20161013,20261012,NaN,Baixo,NaN
122210,122211,2017-12-04,Corporate - Small,NaN,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,BAIRRO DE SOMMERSHIELD,24.0,Certidao da Conservatoria de Regist,20151120ý20171206,20991120ý20991206,CONSERVATORIAýDIRECCAO DE CULTURA,Baixo,NaN


In [4]:
df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in df.columns]
df.columns = df.columns.str.replace('.', '', regex=False)

## Split the data in one part as your reference dataset and the other as your analysis dataset

In [5]:
df_ref = df[~df['customer_since'].str.startswith('2025')]
df_ref

,new_id,customer_since,customer_status,date_of_birth,employment_status,gender,marital_status,placebrth,cust_type,nationality,...,habliter,province,district,legal_doc_name1_id,legal_doc_name1_id_description,legal_iss_date,legal_exp_date,legal_iss_auth,a_m_l_risk_rating,yr_net_monthly_in
0,1,2012-05-23,Private Client - Standard,NaN,NaN,FEMALE,NaN,NaN,NaN,Portugal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevado,NaN
1,2,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medio,NaN
2,3,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medio,NaN
3,4,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevado,NaN
4,5,2013-11-14,Private Client - Standard,2012-07-01,MB,MALE,OTHER,NACALA PORTO,NaN,Mozambique,...,NaN,NAMPULA,NACALA-PORTO,36.0,Licenca/Alvara,20121113,20501113,NAMPULA,Medio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122207,122208,2016-11-29,Private Client - Standard,1973-01-16,EMPLOYED,MALE,DIVORCED,ISRAEL,RETAIL,Israel,...,Curso Superior,CABO DELGADO,MAHATE,2.0,DIRE,20240726ý20250520ý20250520,20250725ý20280519ý20280519,MIGRACAOýMAIRRO MAHATEýREEF INVESTIMENTOS,Baixo,346500.0
122208,122209,2021-07-23,Private Client - Standard,1978-01-10,EMPLOYED,MALE,SINGLE,ZIMBABWE,NaN,Zimbabwe,...,NaN,MAPUTO,MATOLA 'A',3.0,Passaporte,20170614,20270613,REPUBLIC OF ZIMBABWE,Medio,3000.0
122209,122210,2017-12-04,Private Client - Standard,1964-01-16,MB,MALE,DIVORCED,HLATHIKHULU,RETAIL,South Africa,...,NaN,NaN,NaN,3.0,Passaporte,20161013,20261012,NaN,Baixo,NaN
122210,122211,2017-12-04,Corporate - Small,NaN,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,BAIRRO DE SOMMERSHIELD,24.0,Certidao da Conservatoria de Regist,20151120ý20171206,20991120ý20991206,CONSERVATORIAýDIRECCAO DE CULTURA,Baixo,NaN


In [6]:
df_ana = df[df['customer_since'].str.startswith('2025')]
df_ana

,new_id,customer_since,customer_status,date_of_birth,employment_status,gender,marital_status,placebrth,cust_type,nationality,...,habliter,province,district,legal_doc_name1_id,legal_doc_name1_id_description,legal_iss_date,legal_exp_date,legal_iss_auth,a_m_l_risk_rating,yr_net_monthly_in
147,148,2025-02-22,Private Client - Standard,NaN,NaN,NaN,NaN,NaN,NaN,Mozambique,...,NaN,SOFALA,CHAMITE,24.0,Certidao da Conservatoria de Regist,20250220ý20250218,20280820ý20490218,CONSERVATORIA DAS ENTIDADES LEGAISýBAU BEIRA,Baixo,NaN
150,151,2025-02-27,Corporate - Medium,NaN,NaN,NaN,NaN,NaN,PROSPECT,Mozambique,...,NaN,CIDADE DE MAPUTO,MALANGA,29.0,Certidao de Registo / Ministerio da,20250207ý20250213,20250707ý20500213,CONSERVATORIA ENTIDADES LEGAISýBAU,Elevado,NaN
175,176,2025-05-08,Private Client - Standard,NaN,NaN,NaN,NaN,NaN,CORPORATE,Mozambique,...,NaN,CABO DELGADO,PEMBA CIDADE,28.0,Licenca / Alvara para o exercicio d,20250430ý20250425,20490430ý20490425,BAUýCONSERVATORIA DAS ENTIDADES LEGAIS,Elevado,NaN
181,182,2025-02-25,Corporate - Small,NaN,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,MALHANGALAENE 'A',24.0,Certidao da Conservatoria de Regist,20250108ý20250212,20490108ý20490212,MINISTERIO DA JUSTICAýBAU,Medio,NaN
205,206,2025-03-29,Private Client - Standard,NaN,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,CHAMANCULO 'A',24.0,Certidao da Conservatoria de Regist,20250325ý20230713,20991231ý20991231,MINISTERIO DA JUSTICAýBALCAO ATENDIMENTO UNICO,Elevado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122145,122146,2025-05-16,Private Client - Standard,1981-11-11,NaN,FEMALE,OTHER,PORTUGAL,NaN,Mozambique,...,NaN,NaN,NaN,3.0,Passaporte,20201020,20251020,NaN,Elevado,NaN
122166,122167,2025-04-29,Private Client - Standard,1986-02-24,NaN,MALE,SINGLE,BEIRA,RETAIL,Mozambique,...,Bacharelato,SOFALA,BEIRA CIDADE,1.0,BI,20230828,20280827,DIC BEIRA,Elevado,NaN
122191,122192,2025-05-27,Private Client - Standard,1980-07-07,NaN,FEMALE,SINGLE,INHASSUNGE,RETAIL,Mozambique,...,Bacharelato,ZAMBEZIA,MOPEIA,1.0,BI,20240705,20340704,DIC QUELIMANE,Elevado,NaN
122197,122198,2025-03-21,Private Client - Standard,2024-08-10,NaN,FEMALE,SINGLE,MAPUTO,NaN,Mozambique,...,Ate12 ano,CIDADE DE MAPUTO,ALTO MAE 'B',1.0,BI,20250228,20300227,DNIC - MAPUTO,Baixo,NaN


In [ ]:
full_path = os.getcwd()
context = gx.get_context(context_root_dir = full_path.partition('notebooks')[0] + 'gx')

In [8]:
datasource_name = "project_data"
try:
    datasource = context.sources.add_pandas(datasource_name)
except:
    print("Data Source already exists.")
    datasource = context.datasources[datasource_name]

Data Source already exists.


In [9]:
data_asset_name = "project_ana"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe= df_ana)
except:
    print("The data asset alread exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

The data asset alread exists. The required one will be loaded.


In [10]:
data_asset_name = "project_ref"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe= df_ref)
except:
    print("The data asset alread exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

The data asset alread exists. The required one will be loaded.


In [11]:
suite = context.add_or_update_expectation_suite(expectation_suite_name="raw_data_suite")

## Build expectations

In [12]:
# import logging
# from typing import Any, Dict, Tuple

# import numpy as np
# import pandas as pd
# from datetime import datetime
# import re

from great_expectations.core import ExpectationSuite, ExpectationConfiguration

# from pathlib import Path

# from kedro.config import OmegaConfigLoader
# from kedro.framework.project import settings

# conf_path = str(Path('') / settings.CONF_SOURCE)
# conf_loader = OmegaConfigLoader(conf_source=conf_path)
# credentials = conf_loader["credentials"]


# logger = logging.getLogger(__name__)

def build_expectation_suite(expectation_suite_name: str, feature_group: str) -> ExpectationSuite:
    """
    Builder used to retrieve an instance of the validation expectation suite.
    
    Args:
        expectation_suite_name (str): A dictionary with the feature group name and the respective version.
        feature_group (str): Feature group used to construct the expectations.
             
    Returns:
        ExpectationSuite: A dictionary containing all the expectations for this particular feature group.
    """
    
    expectation_suite_bank = ExpectationSuite(
        expectation_suite_name=expectation_suite_name
    )
    

    # numerical features
    if feature_group == 'numerical_features':

        for i in ['no_of_dependents', 'segment_id', 'industry_id', 'legal_doc_name1_id', 'yr_net_monthly_in']:
            expectation_suite_bank.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": i, "type_": "float64"},
                )
            )
        # NewId
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "new_id"}
            )
        )
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": "new_id", "type_": "int64"}
            )
        )
        for i in ['segment_id', 'industry_id', 'legal_doc_name1_id', 'new_id']:
            expectation_suite_bank.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": 1,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )
        for i in ['yr_net_monthly_in', 'no_of_dependents']:
            expectation_suite_bank.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": 0,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )
        for i in ['yr_net_monthly_in', 'no_of_dependents', 'segment_id', 'industry_id', 'legal_doc_name1_id', 'new_id']:
            expectation_suite_bank.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_to_exist",
                    kwargs={"column": i}
                )
            )

    # categorical features
    if feature_group == 'categorical_features':

        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "customer_status", "value_set": ['Private Client - Standard', 'Corporate - Small', 'Corporate - Medium', 'Private Client High Networth', 'Corporate - Large', 'Financial - Large', 'Financial - Small', 'Proprietorship Standard', 'T24 Updates', 'Deceased Individual', 'Partnership firm Standard', 'Financial - Medium', 'Partnership High Networth', 'Customer Deletion', 'Governmental', 'Hotlisted', 'Proprietorship Highnetworth']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "employment_status", "value_set": ['MB', 'LP', 'OTHER', 'EMPLOYED', 'TPE', 'UNEMPLOYED', 'SELF-EMPLOYED', 'RET', 'UE', 'STUDENT', 'RETIRED']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "gender", "value_set": ['FEMALE', 'MALE']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "marital_status", "value_set": ['OTHER', 'DIVORCED', 'SINGLE', 'MARRIED', 'PARTNER', 'WIDOWED']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "cust_type", "value_set": ['RETAIL', 'CORPORATE', 'PROSPECT']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "nationality", "value_set": ['Portugal', 'Mozambique', 'India', 'Pakistan', 'Peoples Republic of China', 'Tokelau', 'South Africa', 'Turkey', 'Netherlands', 'Egypt', 'Switzerland', 'United States of America', 'Congo', 'Yemen', 'Uruguay', 'Afghanistan', 'Korea  Republic of', 'Australia', 'Morocco', 'Somalia', 'Spain', 'Bahrain', 'Lebanon', 'Kenya', 'Mauritius', 'Tanzania  United Republic of', 'Ethiopia', 'Guinea-Bissau', 'Mali', 'Mauritania', 'Guinea', 'Gambia', 'Sierra Leone', 'Mexico', 'Bangladesh', 'Aruba', 'United Arab Emirates', 'Zambia', 'Saudi Arabia', 'Singapore', 'Iran (Islamic Republic of)', 'Nigeria', 'Angola', 'Rwanda', 'Zimbabwe', 'Senegal', 'Hong Kong', 'New Zealand', 'Brazil', 'Italy', 'Cape Verde', 'Philippines', 'Malta', 'Oman', 'Jordan', 'Syrian Arab Republic', 'Libyan Arab Jamahiriya', 'Estado da Palestina', 'Malawi', 'Iraq', 'Great Britain', 'Turks and Caicos Islands', 'Germany', 'Japan', 'Namibia', 'Chile', 'Swaziland', 'Uzbekistan', 'France', 'Luxembourg', 'Norway', 'Bulgaria', 'Croatia', 'Canada', 'Ireland', 'Russian Federation', 'Burundi', 'Czech Republic', 'Uganda', 'Liberia', 'Poland', 'Belgium', 'Benin', 'Greece', 'Kyrgyzstan', 'American Samoa', 'Sao Tome and Principe', 'Tunisia', 'Haiti', 'Columbia', 'Macau', 'Romania', 'Cuba', 'Peru', 'Republic of China (Taiwan)', 'Kazakstan', 'Ghana', 'Finland', 'Puerto Rico', 'Vietnam', 'Israel', 'Thailand', 'Latvia', 'Sri Lanka', 'Nicaragua', 'Congo  Democratic Republic of the', 'Panama', 'Austria', 'Botswana', 'Indonesia', 'Madagascar', 'Comoro Islands', 'Cameroon', 'Nepal', 'Bhutan', 'Sudan', 'Sweden', 'Serbia', 'Mongolia', 'Paraguay', 'Argentina', 'Malaysia', 'Cyprus', 'Honduras', 'Guatemala', 'Monaco', 'Gabon', 'Denmark', 'Eritrea', 'Armenia', 'Ecuador', 'EU Countries', 'Venezuela', 'Dominica', 'Slovenia', 'Antigua And Barbuda', 'Lesotho', 'Georgia', 'Ivory Coast', 'Europa', 'Timor-leste', 'Cayman Islands', 'Northern Mariana Islands', 'San Marino', 'Dominican Republic', 'Chad', 'Algeria', 'Korea  Democratic Peoples Rep. of', 'Iceland', 'Turkmenistan', 'Ukraine', 'Bahamas', 'Bolivia', 'Niger', 'Liechtenstein', 'Hungary', 'Monserrat', 'Jersey', 'Estonia', 'Guadeloupe', 'Costa Rica', 'Lithuania', 'Guyana', 'Qatar', 'Andorra', 'Reunion', 'Yugoslavia', 'Belarus', 'Slovakia', 'Saint Lucia', 'Papua New Guinea', 'Barbados', 'Azerbaijan', 'Fiji', 'Holy See (Vatican City State)', 'Central African Republic']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "residence_code", "value_set": ['MZ', 'PT', 'ZA', 'NL', 'EG', 'CH', 'CN', 'US', 'IN', 'YE', 'UY', 'AF', 'KR', 'AU', 'ES', 'BH', 'LB', 'MU', 'TZ', 'KE', 'AW', 'AE', 'TR', 'AO', 'NG', 'ZM', 'RW', 'BR', 'ZW', 'MA', 'MR', 'IT', 'NZ', 'MT', 'OM', 'JP', 'CL', 'SZ', 'SA', 'FR', 'LU', 'HR', 'PL', 'GR', 'KG', 'ST', 'TN', 'IR', 'IE', 'GB', 'MW', 'VN', 'JO', 'FI', 'SE', 'PK', 'LV', 'BE', 'CV', 'CU', 'PA', 'MX', 'CO', 'DE', 'AT', 'BW', 'ID', 'MG', 'CA', 'IQ', 'HK', 'KM', 'CM', 'NO', 'CD', 'SG', 'RO', 'RS', 'GH', 'PY', 'PH', 'TW', 'LS', 'MY', 'BM', 'CZ', 'MC', 'AR', 'DK', 'PE', 'XE', 'SI', 'AG', 'TH', 'UG', 'RE', 'GE', 'GM', 'CI', 'KZ', 'CY', 'GA', 'GN', 'MO', 'KP', 'TC', 'IL', 'MN', 'BS', 'UA', 'RU', 'MP', 'HU', 'LR', 'EE', 'AM', 'CR', 'BD', 'SO', 'AD', 'SN', 'SK', 'LC', 'AS', 'BB', 'LT', 'BI', 'LK', 'ML', 'ET', 'EU', 'ER']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "residence_status", "value_set": ['HOME.OWNER', 'OTHER', 'TENANT', 'LIVING.WTH.PARENTS', 'SQUATTER']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "residence_type", "value_set": ['RESIDENTIAL.APT', 'INDEPEDENT.HOUSE', 'FARM.HOUSE', 'SERVICED.APT']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "seg_group", "value_set": ['Company', 'Personal']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "title", "value_set": ['MRS', 'MR', 'DR', 'DRS', 'MISS', 'MAST1', 'ENG', 'PHD', 'MAST']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "cust_type1", "value_set": ['RETAIL', 'CORPORATE', 'PROSPECT']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "habliter", "value_set": ['Ate12 ano', 'Curso Superior', 'Bacharelato', 'Doutoramento', 'Mestrado', 'S.Estudos', 'Ensino Primario']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "province", "value_set": ['NAMPULA', 'CIDADE DE MAPUTO', 'MAPUTO', 'SOFALA', 'TETE', 'CABO DELGADO', 'NOT APPLICABLE', 'MANICA', 'ZAMBEZIA', 'NIASSA', 'INHAMBANE', 'GAZA']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "legal_doc_name1_id_description", "value_set": ['Licenca/Alvara', 'DIRE', 'Certidao da Conservatoria de Regist', 'Licenca / Alvara para o exercicio d', 'BI', 'Certidao de reserva de nome', 'Passaporte', 'Others (specify on free text field)', 'Certidao de Registo Comercial (Caso', 'Acta(s) da Assembleia Geral dos Acc', 'Escritura Publica da constituicao d', 'Certidao de Registo / Ministerio da', 'NUIT- Numero Unico de Identificacao', 'Cedula Pessoal', 'Recibo de pedido de BI', 'Cartao de identificacao de refugiad', 'NUIT-Numero Unico de Identificac', 'Carta de conducao', 'BR com o Pacto Social / Estatutos d', 'Termo de Autorizacao do Ministerio"', 'Certidao narrativa completa de nasc', 'Certidao da sentenca do Tribunal de', 'Numero de Registo Fiscal ( Modelo 4', 'Cartao de recenseamento eleitoral', 'Prova de Residencia', 'Prova documental de nao residente', 'Cartao de agricultor/trabalhador ab', 'Cartao de INSS', 'Documento de Transitario', 'Declaracao Comprovativa / Ministeri', 'Cedula militar', 'Documento de Regime Especial', 'Contrato de Trabalho', 'Documento de Embaixada', 'Procuracoes (especificar)', 'Documento de Staff Embaixada', 'Acta(s) da reuniao dos socios (pode', 'Documeto de Exportador']},
            )
        ) 
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_distinct_values_to_be_in_set",
                kwargs={"column": "a_m_l_risk_rating", "value_set": ['Elevado', 'Medio', 'Baixo']},
            )
        ) 
        for i in ['placebrth', 'ocupation_desc', 'town_country', 'district', 'legal_iss_auth', 'legal_exp_date', 'date_of_birth', 'birth_in_corp_date', 'legal_iss_date', 'customer_since']:
            expectation_suite_bank.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": i, "type_": "object"},
                )
            )
     
    return expectation_suite_bank

In [13]:
validation_expectation_suite_numerical = build_expectation_suite("numerical_expectations_raw", "numerical_features")
validation_expectation_suite_categorical = build_expectation_suite("categorical_expectations_raw", "categorical_features")

In [14]:
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_numerical)
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_categorical)

{
  "expectation_suite_name": "categorical_expectations_raw",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
        "column": "customer_status",
        "value_set": [
          "Private Client - Standard",
          "Corporate - Small",
          "Corporate - Medium",
          "Private Client High Networth",
          "Corporate - Large",
          "Financial - Large",
          "Financial - Small",
          "Proprietorship Standard",
          "T24 Updates",
          "Deceased Individual",
          "Partnership firm Standard",
          "Financial - Medium",
          "Partnership High Networth",
          "Customer Deletion",
          "Governmental",
          "Hotlisted",
          "Proprietorship Highnetworth"
        ]
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
        "column": "employment_status",

## Analysis

In [15]:
def get_validation_results(checkpoint_result):
    # validation_result is a dictionary containing one key-value pair
    validation_result_key, validation_result_data = next(iter(checkpoint_result["run_results"].items()))

    # Accessing the 'actions_results' from the validation_result_data
    validation_result_ = validation_result_data.get('validation_result', {})

    # Accessing the 'results' from the validation_result_data
    results = validation_result_["results"]
    meta = validation_result_["meta"]
    use_case = meta.get('expectation_suite_name')
    
    
    df_validation = pd.DataFrame({},columns=["Success","Expectation Type","Column","Column Pair","Max Value",\
                                       "Min Value","Element Count","Unexpected Count","Unexpected Percent","Value Set","Unexpected Value","Observed Value"])
    
    
    for result in results:
        # Process each result dictionary as needed
        success = result.get('success', '')
        expectation_type = result.get('expectation_config', {}).get('expectation_type', '')
        column = result.get('expectation_config', {}).get('kwargs', {}).get('column', '')
        column_A = result.get('expectation_config', {}).get('kwargs', {}).get('column_A', '')
        column_B = result.get('expectation_config', {}).get('kwargs', {}).get('column_B', '')
        value_set = result.get('expectation_config', {}).get('kwargs', {}).get('value_set', '')
        max_value = result.get('expectation_config', {}).get('kwargs', {}).get('max_value', '')
        min_value = result.get('expectation_config', {}).get('kwargs', {}).get('min_value', '')

        element_count = result.get('result', {}).get('element_count', '')
        unexpected_count = result.get('result', {}).get('unexpected_count', '')
        unexpected_percent = result.get('result', {}).get('unexpected_percent', '')
        observed_value = result.get('result', {}).get('observed_value', '')
        if type(observed_value) is list:
            #sometimes observed_vaue is not iterable
            unexpected_value = [item for item in observed_value if item not in value_set]
        else:
            unexpected_value=[]
        
        df_validation = pd.concat([df_validation, pd.DataFrame.from_dict( [{"Success" :success,"Expectation Type" :expectation_type,"Column" : column,"Column Pair" : (column_A,column_B),"Max Value" :max_value,\
                                           "Min Value" :min_value,"Element Count" :element_count,"Unexpected Count" :unexpected_count,"Unexpected Percent":unexpected_percent,\
                                                  "Value Set" : value_set,"Unexpected Value" :unexpected_value ,"Observed Value" :observed_value}])], ignore_index=True)
        
    return df_validation

In [16]:
data_asset_analysis = datasource.get_asset("project_ref")

In [20]:
batch_request = data_asset_analysis.build_batch_request(df_ref)

In [21]:
checkpoint_cat = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_categorical",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "categorical_expectations_raw",
        },
    ],
)
checkpoint_result = checkpoint_cat.run()

Calculating Metrics:   0%|          | 0/19 [00:00<?, ?it/s]

In [22]:
df_validation = get_validation_results(checkpoint_result)
df_validation

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_distinct_values_to_be_in_set,customer_status,"(, )",,,,,,"[Private Client - Standard, Corporate - Small,...",[],"[Corporate - Large, Corporate - Medium, Corpor..."
1,True,expect_column_distinct_values_to_be_in_set,employment_status,"(, )",,,,,,"[MB, LP, OTHER, EMPLOYED, TPE, UNEMPLOYED, SEL...",[],"[EMPLOYED, LP, MB, OTHER, RET, RETIRED, SELF-E..."
2,True,expect_column_distinct_values_to_be_in_set,gender,"(, )",,,,,,"[FEMALE, MALE]",[],"[FEMALE, MALE]"
3,True,expect_column_distinct_values_to_be_in_set,marital_status,"(, )",,,,,,"[OTHER, DIVORCED, SINGLE, MARRIED, PARTNER, WI...",[],"[DIVORCED, MARRIED, OTHER, PARTNER, SINGLE, WI..."
4,True,expect_column_distinct_values_to_be_in_set,cust_type,"(, )",,,,,,"[RETAIL, CORPORATE, PROSPECT]",[],"[CORPORATE, PROSPECT, RETAIL]"
5,True,expect_column_distinct_values_to_be_in_set,nationality,"(, )",,,,,,"[Portugal, Mozambique, India, Pakistan, People...",[],"[Afghanistan, Algeria, American Samoa, Andorra..."
6,True,expect_column_distinct_values_to_be_in_set,residence_code,"(, )",,,,,,"[MZ, PT, ZA, NL, EG, CH, CN, US, IN, YE, UY, A...",[],"[AD, AE, AF, AG, AM, AO, AR, AS, AT, AU, AW, B..."
7,True,expect_column_distinct_values_to_be_in_set,residence_status,"(, )",,,,,,"[HOME.OWNER, OTHER, TENANT, LIVING.WTH.PARENTS...",[],"[HOME.OWNER, LIVING.WTH.PARENTS, OTHER, SQUATT..."
8,True,expect_column_distinct_values_to_be_in_set,residence_type,"(, )",,,,,,"[RESIDENTIAL.APT, INDEPEDENT.HOUSE, FARM.HOUSE...",[],"[FARM.HOUSE, INDEPEDENT.HOUSE, RESIDENTIAL.APT..."
9,True,expect_column_distinct_values_to_be_in_set,seg_group,"(, )",,,,,,"[Company, Personal]",[],"[Company, Personal]"


In [25]:
df_validation[df_validation.Success == False].empty

True

In [70]:
checkpoint_num = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_categorical",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "numerical_expectations_raw",
        },
    ],
)
checkpoint_result = checkpoint_num.run()

Calculating Metrics:   0%|          | 0/48 [00:00<?, ?it/s]

In [71]:
df_validation = get_validation_results(checkpoint_result)
df_validation

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_values_to_be_of_type,no_of_dependents,"(, )",,,,,,,[],float64
1,True,expect_column_values_to_be_between,no_of_dependents,"(, )",None,0,112451,0,0.0,,[],
2,True,expect_column_to_exist,no_of_dependents,"(, )",,,,,,,[],
3,True,expect_column_values_to_be_of_type,segment_id,"(, )",,,,,,,[],float64
4,True,expect_column_values_to_be_between,segment_id,"(, )",None,1,112451,0,0.0,,[],
5,True,expect_column_to_exist,segment_id,"(, )",,,,,,,[],
6,True,expect_column_values_to_be_of_type,industry_id,"(, )",,,,,,,[],float64
7,True,expect_column_values_to_be_between,industry_id,"(, )",None,1,112451,0,0.0,,[],
8,True,expect_column_to_exist,industry_id,"(, )",,,,,,,[],
9,True,expect_column_values_to_be_of_type,legal_doc_name1_id,"(, )",,,,,,,[],float64


In [72]:
df_validation[df_validation.Success == False]


,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
